In [1]:
import json
import torch

state = torch.load('models/ddqn/reproducibility-0_1wpkgfkj_327679_20220130T222047.pth')
print(json.dumps(state['config'], indent=2, sort_keys=True))

{
  "activation": "relu",
  "alpha": 0.6,
  "batch_size": 1024,
  "buffer_size": 300000,
  "buffer_type": "prioritized",
  "embed": false,
  "embed_size": 0,
  "env": "CartPole-v0",
  "env_args": {},
  "eps_sched_len": 200000,
  "gamma": 0.995,
  "layer_size": 64,
  "learning_starts": 2048,
  "log_step": 8192,
  "lr": 0.004,
  "max_episode_steps": null,
  "num_layers": 3,
  "save_final": true,
  "save_max_eps": false,
  "seed": 0,
  "steps": 327680,
  "target_update_freq": 8192,
  "training_freq": 64
}


In [3]:
import gym
from spin_class.algos.ddqn import make_model

kwargs = state['config']['env_args'] if 'env_args' in state['config'] else {}
env = gym.make(state['config']['env'], **kwargs)
device = torch.device('cpu')
q_net = make_model(env, device, state['config'])
q_net.load_state_dict(state['q_state_dict'])
q_net.eval()

DQNMLP(
  (head): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [4]:
import gym
from gym import wrappers
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from torch.distributions.normal import Normal

# start virtual display
if 'display' not in globals():
    display = Display(visible=False, size=(1400, 900))
    display.start()

def play(q_net, env, steps=1000):
    env = wrappers.Monitor(env, "./video", force=True)
    obs_dtype = (
        torch.int64
        if isinstance(env.observation_space, gym.spaces.Discrete)
        else torch.float32
    )
    obs = env.reset()
    for _ in range(steps):
        with torch.no_grad():
            obs_t = torch.as_tensor(obs, dtype=obs_dtype, device=device).unsqueeze(
                    0
                )
            q = q_net(obs_t)[0]
            action = torch.argmax(q).cpu().numpy().tolist()
        obs, reward, done, info = env.step(action)
        if done:
            print(_)
            break
    env.close()

    video = io.open('./video/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''
        <video alt="test" autoplay loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>'''.format(encoded.decode('ascii'))))

    #HTML(data='''
    #    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>
    #'''.format(encoded.decode('ascii')))

In [7]:
play(q_net, env, steps=200)

199
